In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = sns.load_dataset('iris')

In [3]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [4]:
df['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df.drop('species',axis=1)
y = df['species']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
model_knn = KNeighborsClassifier(n_neighbors = 5)

In [12]:
model_knn.fit(X_train, y_train)

KNeighborsClassifier()

In [14]:
model_knn.score(X_test, y_test)

0.98

In [20]:
from sklearn.svm import SVC
model_svm = SVC(gamma='auto')

In [21]:
model_svm.fit(X_train, y_train)

SVC(gamma='auto')

In [22]:
model_svm.score(X_test, y_test)

1.0

will use grid search cv now

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
classifier = GridSearchCV((model_svm), {
    'C':[1,10,20,30],
    'kernel':['rbf','linear']
}, cv=5, return_train_score = False)

In [25]:
classifier.fit(X,y) #train test split not required in cross validation

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20, 30], 'kernel': ['rbf', 'linear']})

In [27]:
classifier.cv_results_

{'mean_fit_time': array([0.00323448, 0.00209904, 0.00210028, 0.00206766, 0.00214577,
        0.00221395, 0.00209932, 0.00213785]),
 'std_fit_time': array([5.34909021e-04, 5.55725017e-05, 7.26076607e-05, 5.38645773e-05,
        1.02586614e-04, 1.46725281e-04, 3.86071464e-05, 1.81928805e-04]),
 'mean_score_time': array([0.00212164, 0.00150819, 0.00147476, 0.00201349, 0.00157213,
        0.001579  , 0.00148263, 0.0014493 ]),
 'std_score_time': array([2.37127211e-04, 9.84151233e-05, 7.56147972e-05, 8.37128861e-04,
        1.76236911e-04, 9.58980343e-05, 5.19513154e-05, 6.04396318e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_valu

In [28]:
results =  pd.DataFrame(classifier.cv_results_)

In [29]:
results[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667
6,30,rbf,0.960000
7,30,linear,0.960000


In [30]:
from sklearn.model_selection import RandomizedSearchCV

In [31]:
classifier_r = RandomizedSearchCV((model_svm),{
    'C' : [1,10,20,30],
    'kernel' :['rbf','linear'],
},n_iter = 4,cv = 5,return_train_score = False)

In [32]:
classifier_r.fit(X,y)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=4,
                   param_distributions={'C': [1, 10, 20, 30],
                                        'kernel': ['rbf', 'linear']})

In [33]:
results = pd.DataFrame(classifier_r.cv_results_)

In [34]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002671,0.000378,0.001951,0.000558,linear,10,"{'kernel': 'linear', 'C': 10}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,2
1,0.002217,0.000250,0.001564,0.000119,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002007,0.000022,0.001421,0.000009,linear,20,"{'kernel': 'linear', 'C': 20}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,4
3,0.002220,0.000149,0.001503,0.000017,rbf,20,"{'kernel': 'rbf', 'C': 20}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3


In [35]:
results[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,1,linear,0.980000
2,20,linear,0.966667
3,20,rbf,0.966667
